### 1. mon_feature.pkl upload

In [1]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = '/content/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER2 = [
    'total_transmission_time', 'std_inter_packet_time',
    'num_outgoing_packets', 'avg_incoming_burst_size',
    'cumul_packets_30pct', 'cumul_packets_10pct',
    'incoming_order_skew', 'outgoing_order_skew',
    'cumul_max', 'bigram_OO', 'avg_outgoing_order_first_30',
    'num_incoming_first_30', 'incoming_packet_ratio'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER2]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)

       total_transmission_time  std_inter_packet_time  num_outgoing_packets  \
0                        10.14               0.041168                 121.0   
1                        10.16               0.163930                  80.0   
2                        11.11               0.066661                 118.0   
3                        13.36               0.047809                 122.0   
4                        10.64               0.038760                 115.0   
...                        ...                    ...                   ...   
18995                    43.91               0.143962                 619.0   
18996                    15.60               0.019465                 552.0   
18997                    14.93               0.016411                 579.0   
18998                    19.91               0.033281                 690.0   
18999                    13.76               0.011074                 757.0   

       avg_incoming_burst_size  cumul_packets_30pct

### 2. XGBOOST

#### (1) install

In [2]:
!pip install xgboost

In [3]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#### (2) Split dataset

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### (3) XGBoost 다중분류 모델 정의

In [5]:
model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    num_class=len(np.unique(y)),
    eval_metric='mlogloss',
    random_state=42
)

#### (4) 모델 학습

In [6]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None, num_class=95, ...)

#### (5) 예측

In [7]:
y_pred = model.predict(X_test)

#### (6) 평가

In [8]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8692105263157894

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.93      0.87        42
           1       0.97      0.76      0.85        42
           2       0.89      0.94      0.92        35
           3       0.93      0.97      0.95        29
           4       0.94      0.77      0.85        39
           5       0.98      0.96      0.97        45
           6       0.88      0.95      0.91        44
           7       0.72      0.81      0.76        36
           8       0.92      0.71      0.80        34
           9       0.79      0.74      0.77        31
          10       0.93      0.91      0.92        47
          11       0.92      0.94      0.93        35
          12       0.90      0.90      0.90        42
          13       0.80      0.80      0.80        40
          14       0.89      0.86      0.87        36
          15       0.75      0.86      0.80        35
          16       0.95    